In [1]:
import sys
import os

sys.path.append(os.path.abspath("src"))  # Adjust the path based on your structure


In [2]:
!pip install pyyaml

import yaml


In [3]:
%pwd

'c:\\Users\\91844\\Desktop\\deep_leaning_project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\91844\\Desktop\\deep_leaning_project'

In [6]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [7]:
!pip install joblib


In [8]:
import joblib

In [15]:
import sys
import os

sys.path.append(os.path.abspath("src"))


In [17]:
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.constants import *

In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [26]:
import os
import zipfile
import gdown
from cnnClassifier.utils.common import get_size
from cnnClassifier import logger

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        """
        Downloads a file from a Google Drive URL and saves it locally.
        Returns the local file path.
        """
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file

            os.makedirs(os.path.dirname(zip_download_dir), exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?export=download&id='
            gdown.download(prefix + file_id, zip_download_dir, quiet=False)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
            return zip_download_dir

        except Exception as e:
            logger.error(f"Error in downloading file: {e}")
            raise e

    def extract_zip_file(self) -> None:
        """
        Extracts the downloaded ZIP file into a specified directory.
        """
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)

            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)

            logger.info(f"Extracted zip file {self.config.local_data_file} to {unzip_path}")

        except Exception as e:
            logger.error(f"Error in extracting zip file: {e}")
            raise e


In [25]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-11 19:21:51,092: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-11 19:21:51,099: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-11 19:21:51,104: INFO: common: created directory at: artifacts]
[2025-07-11 19:21:51,106: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-11 19:21:51,108: INFO: 4067539729: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=3b7f4253-aaf5-4053-8d70-c490699ed44d
To: c:\Users\91844\Desktop\deep_leaning_project\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [02:04<00:00, 464kB/s] 

[2025-07-11 19:24:01,318: INFO: 4067539729: Downloaded data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]


[2025-07-11 19:24:02,536: INFO: 4067539729: Extracted zip file artifacts/data_ingestion/data.zip to artifacts/data_ingestion]
